In [712]:
import pickle

## 标签映射错误

In [ ]:
## 保存 vit-b/32 k400的表示

In [713]:
with open('../lists/k400_cls_feas512_vitb_32','rb') as f:
    k400=pickle.load(f)

In [784]:
with open('../lists/hmdb51_cls_feas512_vitb_32','rb') as f:
    hmdb51=pickle.load(f)

In [907]:
with open('../test_records/ucf101/ViT-B/32/clip_before_one_temporal/video_features','rb') as f:
    video_hmdb51=pickle.load(f)

In [908]:
with open('../test_records/ucf101/ViT-B/32/clip_before_one_temporal/pre_labels','rb') as f:
    labels_hmdb51=pickle.load(f)

In [696]:
with open('../test_records/k400/ViT-B/32/k400_visual_center/visual_center','rb') as f:
    k400_visual_center=pickle.load(f)

In [790]:
hmdb51_norm=hmdb51/hmdb51.norm(dim=-1,keepdim=True)

In [804]:
hmdb51_norm[31].topk(10)

torch.return_types.topk(
values=tensor([0.5413, 0.5406, 0.1030, 0.0838, 0.0748, 0.0739, 0.0635, 0.0619, 0.0609,
        0.0606]),
indices=tensor([133, 312, 493, 441, 143, 479,  43,  73,  67, 311]))

In [814]:
inds=(-4.5*video_hmdb51[0]).topk(10).indices

In [817]:
-(-4.5*video_hmdb51[0]).topk(10).values

tensor([-1.3921, -0.7358, -0.3985, -0.3935, -0.3875, -0.3786, -0.3602, -0.3405,
        -0.3383, -0.3238])

In [815]:
hmdb51_norm[31][inds]

tensor([-0.1044, -0.0504, -0.0086, -0.0029, -0.0505,  0.0117, -0.0244,  0.0245,
        -0.1046, -0.0611])

In [810]:
(4.5*video_hmdb51[0]@hmdb51_norm.T).softmax(dim=-1).topk(3)

torch.return_types.topk(
values=tensor([0.0353, 0.0233, 0.0228]),
indices=tensor([31, 32, 48]))

In [ ]:
test_video=video_hmdb51[1][video_hmdb51[1]<0]

In [862]:
labels_hmdb51[2]

tensor([31, 30, 48, 28, 38, 31])

In [873]:
video_fea=video_hmdb51.unsqueeze(1)

In [874]:
video_fea.shape

torch.Size([6766, 1, 512])

In [876]:
hmdb51_norm1=hmdb51_norm.unsqueeze(0)

In [887]:
sumsame=torch.sum(((hmdb51_norm1>0) & (video_fea>0)) | ((hmdb51_norm1<0) & (video_fea<0)),dim=-1).topk(1).indices

In [869]:
indices=(4.6*video_hmdb51@hmdb51_norm.T).softmax(dim=-1).topk(1).indices

In [870]:
hmdb_true_label=labels_hmdb51[:,-1]
# ucf101_true_label=labels_ucf101[:,-1]
hmdb_true_label=hmdb_true_label.unsqueeze(dim=1)

In [895]:
trueratio=sum((sumsame==hmdb_true_label)&(indices==hmdb_true_label)).numpy()[0]/video_hmdb51.shape[0]

In [896]:
trueratio

0.3151049364469406

In [796]:
labels_hmdb51[0]

tensor([31, 32, 48, 49, 13, 31])

In [726]:
def gettest_train(test_datas,train_datas):
    '''
    为测试集生成任务相关的训练文本指引
    '''
    test_datas/=test_datas.norm(dim=1,keepdim=True)
    train_datas/=train_datas.norm(dim=1,keepdim=True)
    similarity=(100*test_datas@train_datas.T).softmax(dim=-1)
    values=similarity.softmax(dim=-1).topk(3).values.softmax(dim=-1)
    indices=similarity.softmax(dim=-1).topk(3).indices
#     return indices
    return train_datas[indices].mean(dim=1)

In [905]:
coltrain_hmdb51=gettest_train(ucf101_in_gr,k400)

In [909]:
hmdb_true_label=labels_hmdb51[:,-1]
# ucf101_true_label=labels_ucf101[:,-1]
hmdb_true_label=hmdb_true_label.unsqueeze(dim=1)

In [759]:
def get_accuracy(video_feas,cls_feas,coltrain,true_las,ratio):
#     coltrain=k400_visual_center[coltrain].mean(dim=1)
    simi_visual=(100*video_feas@coltrain.T).softmax(dim=-1)
    similarity=(100*video_feas@cls_feas.T).softmax(dim=-1)
    all_simi=ratio*simi_visual+(1-ratio)*similarity
    indices=all_simi.softmax(dim=-1).topk(1).indices
    return indices

In [911]:
x=[]
res_hmdb51=[]
# res_ucf101=[]
for i in range(0,11):
    ratio=i/10
    x.append(ratio)
#     print(ratio)
    indices=get_accuracy(video_hmdb51,ucf101_in_gr,coltrain_hmdb51,hmdb_true_label,ratio)
    trueratio=sum(indices==hmdb_true_label).numpy()[0]/video_hmdb51.shape[0]
    res_hmdb51.append(trueratio)
    
#     indices=get_accuracy(video_ucf101,ucf101,coltrain_ucf101,ucf101_true_label,ratio)
#     trueratio=sum(indices==ucf101_true_label).numpy()[0]/video_ucf101.shape[0]
#     res_ucf101.append(trueratio)

In [912]:
res_hmdb51

[0.7774932208496536,
 0.7781711358843025,
 0.777342573064176,
 0.772597167821633,
 0.7676257909008738,
 0.7594908104850858,
 0.7417143717987346,
 0.7195691473335342,
 0.6794215125037661,
 0.6301596866526062,
 0.5530280204880988]

In [468]:
res_ucf101

[0.7110575474540525,
 0.7110575474540525,
 0.7114341669177463,
 0.7122627297378729,
 0.7121120819523953,
 0.713919855378126,
 0.513859596263935,
 0.4212865320879783,
 0.3706688761675203,
 0.34829768002410366,
 0.33052124133775235]

In [145]:
import torch

In [146]:
a=torch.randn((4,8,512))
a[:,0,:].shape

torch.Size([4, 512])

In [147]:
position_ids = torch.arange(8, dtype=torch.long)
position_ids

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [148]:
frame_position_embeddings = torch.nn.Embedding(77, 512)

In [149]:
frame_position_embeddings(position_ids).shape

torch.Size([8, 512])

In [150]:
frame_position_embeddings

Embedding(77, 512)

In [405]:
import os
num=sorted(os.listdir('../../UCF_101_frame_org/'))

In [406]:
len(num)

101

In [407]:
k400acs=[i.strip().split(',')[1] for i in open('../lists/ucf101_action_gra.txt').readlines()]

In [408]:
for i in range(101):
    ac=k400acs[i]
    ac=ac.replace(' ','_')
    nu=num[i].replace(' ','_')
    if nu!=ac:
        print(nu,ac,i)
        

In [182]:
scale=512 ** -0.5

In [183]:
class_embedding=torch.nn.Parameter(scale*torch.randn(512))

In [185]:
class_embedding.shape

torch.Size([512])

In [187]:
x=torch.randn((2,8,512))

In [192]:
x[:,0,:].shape

torch.Size([2, 512])

In [193]:
 torch.zeros(2, 1,512)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])

In [194]:
class_embedding+torch.zeros(2, 1,512)

tensor([[[-0.0104, -0.0285, -0.0653,  ..., -0.0244, -0.0131,  0.0068]],

        [[-0.0104, -0.0285, -0.0653,  ..., -0.0244, -0.0131,  0.0068]]],
       grad_fn=<AddBackward0>)

In [199]:
a=torch.randn((2,512))
a=torch.unsqueeze(a,dim=1)
a.shape

torch.Size([2, 1, 512])

In [196]:
x=torch.randn((2,8,512))

## 构造不同的prompt

In [2]:
import sys 
sys.path.append("..") 
import clip
import torch
import pickle
clip_model, clip_state_dict = clip.load('ViT-B/32',
                             device='cpu',jit=False,
                             internal_modeling=False,
                             T=8,
                             dropout=0.0,
                            emb_dropout=0.0,
                            pretrain=True)

dropout used:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
dropout used:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
loading clip pretrained model!


In [4]:
from PIL import Image

In [ ]:
'../25_af.png'

In [8]:
image=Image.open('../25_af.png').convert('RGB')

In [30]:
import torch.nn as nn
class Textencoder(nn.Module):
    def __init__(self,clip_model) :
        super().__init__()
        # self.vocab_size=clip_model.vocab_size
        self.transformer=clip_model.transformer
        self.positional_embedding=clip_model.positional_embedding
        self.emb_dropout = clip_model.emb_dropout
        self.text_projection =clip_model.text_projection
        self.ln_final=clip_model.ln_final
    def encode_text(self, prompts,tokenized_prompts):
        '''
        prompts: 为prompt+class_name token_embedding的结果
        tokenized_prompts：为prompt+class_name 分词的结果
        '''
        # x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]

        x = prompts + self.positional_embedding.type(self.dtype)
        if self.emb_dropout > 0:
            x = self.dropout(x)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)

        # x.shape = [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(dim=-1)] @ self.text_projection

        return x 

In [31]:
textual=Textencoder(clip_model)

In [34]:
# for name,param in textual.named_parameters():
#     print(name)
# clip_model.ln_final.weight.shape

torch.Size([512])

In [35]:
clip_model.token_embedding

Embedding(49408, 512)

In [14]:
def text_prompt_test(data):
    text_aug = ['a video of a person {}.']
    text_dict={}
    num_text_aug=len(text_aug)
    for ii,txt in enumerate(text_aug):
        text_dict[ii]=torch.cat([clip.tokenize(txt.format(c)) for c in data])
    feature=text_dict[0]
    return feature

In [15]:
import numpy as np
## 获取在知识图谱中的类别
k400_acs=[i.strip().split(',')[1] for i in open('../lists/ucf101_action_gra.txt').readlines()]
k400_acs=np.array(k400_acs)
token1=text_prompt_test(k400_acs)


In [16]:
with torch.no_grad():
    ucf101_in_gr=clip_model.encode_text(token1)

In [17]:
ucf101_in_gr/=ucf101_in_gr.norm(dim=-1,keepdim=True)

In [18]:
ucf101_in_gr[0][:5]

tensor([ 0.0022,  0.0148,  0.0101,  0.0300, -0.0107])

In [920]:
f=open('../lists/ucf101_cls_feas512_vitl_14_org','wb')
pickle.dump(ucf101_in_gr,f)
f.close()

In [778]:
def gettest_train_str(test_datas,train_datas):
    '''
    为训练集的类别选择最为相似的5个类别
    '''
    test_datas/=test_datas.norm(dim=1,keepdim=True)
    train_datas/=train_datas.norm(dim=1,keepdim=True)
    similarity=(100*test_datas@train_datas.T).softmax(dim=-1)
#     values=similarity.softmax(dim=-1).topk(3).values.softmax(dim=-1)
    indices=similarity.softmax(dim=-1).topk(3).indices.numpy()
#     return indices
#     strr=','.join
    colacs=k400_acs[indices]
    re=[]
    for ac in colacs:
        re.append(','.join(ac))
    return re

In [779]:
colacsstr=gettest_train_str(hmdb51,k400)

In [780]:
colacsstr_token=text_prompt_test(colacsstr)

In [781]:
with torch.no_grad():
    cls1=clip_model.encode_text(colacsstr_token)

In [261]:
a= ['clapping', 'applauding', 'pumping fist']
','.join(a)

'clapping,applauding,pumping fist'

In [ ]:
### hmdb51 选择前5个prompt会有一定的提升

In [415]:
k400

tensor([[ 0.0281,  0.0211, -0.0110,  ..., -0.0354, -0.0097, -0.0311],
        [ 0.0748,  0.0356, -0.0177,  ..., -0.0302,  0.0383,  0.0285],
        [ 0.0467,  0.0272,  0.0116,  ...,  0.0299,  0.0157, -0.0400],
        ...,
        [ 0.0223, -0.0224, -0.0048,  ..., -0.0474,  0.0395,  0.0114],
        [ 0.0336,  0.0019, -0.0107,  ..., -0.0359, -0.0388, -0.0034],
        [ 0.0273,  0.0202, -0.0116,  ..., -0.0273, -0.0012,  0.0204]])

In [599]:
labels=[i.strip().split(',')[0] for i in open('../lists/ucf101_action_gra.txt').readlines()]
or_labels=[i.strip().split(',')[1] for i in open('../lists/ucf101_labels.txt').readlines()]

In [447]:
frames_la=[i for i in  sorted(os.listdir('../../UCF_101_frame_org/'))]

In [448]:
for i in range(len(labels)):
    if frames_la[i]!=or_labels[i]:
        print(frames_la[i],or_labels[i])

In [452]:
f=open('../lists/ucf101_action_gra.txt','a')
for la in labels:
    la=la.replace(' ',',')
    f.write(la+'\n')
f.close()    

In [550]:

labels=[i.strip() for i in open('../lists/k600_labels.txt').readlines()]
f=open('../lists/k600_labels_test.txt','a')
for ind,la in enumerate(labels):
    f.write(str(ind)+','+la+'\n')
f.close()

In [732]:
a=torch.randn((2,8,512))
b=torch.randn((2,8,512))

In [733]:
a=torch.max(a,dim=1).values
b=torch.max(b,dim=1).values

In [738]:
torch.max(a,b)[:,:5]

tensor([[1.6476, 0.9748, 2.4323, 1.9777, 1.6841],
        [2.3086, 1.6194, 2.4260, 1.0982, 2.8340]])

In [739]:
a[:,:5]

tensor([[1.6476, 0.9748, 1.6356, 1.7956, 1.6841],
        [1.8022, 1.6194, 1.0366, 0.6992, 2.8340]])

In [19]:
import os

In [20]:
k400=[i.strip().split(',')[1] for i in open('../lists/kinetics_400_labels.txt')]

In [24]:
for ind,name in enumerate(sorted(os.listdir('../../kinetics400_frames/'))):
    a=k400[ind].replace(' ','_')
    if a!=name: 
        print(k400[ind],name,ind)

In [ ]:
## KL散度

## 文本端的prompt tuning

In [1]:
k400_acs=[i.strip().split(',')[1] for i in open('../lists/kinetics_400_labels.txt').readlines()]

In [2]:
n_ctx=8
prompt_prefix=" ".join(["X"]*n_ctx)

In [5]:
prompts=[prompt_prefix + " " + name +"." for name in k400_acs] 

In [7]:
tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts])

In [8]:
tokenized_prompts

tensor([[49406,   343,   343,  ...,     0,     0,     0],
        [49406,   343,   343,  ...,     0,     0,     0],
        [49406,   343,   343,  ...,     0,     0,     0],
        ...,
        [49406,   343,   343,  ...,     0,     0,     0],
        [49406,   343,   343,  ...,     0,     0,     0],
        [49406,   343,   343,  ...,     0,     0,     0]])

In [10]:
with torch.no_grad():
    embedding = clip_model.token_embedding(tokenized_prompts)

In [12]:
embedding.shape

torch.Size([400, 77, 512])

In [13]:
token_suffix=embedding[:,1+n_ctx:,:] # 类别和结束符

In [17]:
token_prefix=embedding[:,:1,:]

In [20]:
ctx_vectors = torch.empty(400, n_ctx, 512)

In [21]:
ctx_vectors=torch.nn.init.normal_(ctx_vectors, std=0.02)

In [23]:
prompts=torch.cat([token_prefix,ctx_vectors,token_suffix],dim=1)

In [26]:
prompts[0,:5,0]

tensor([-0.0017,  0.0085,  0.0296,  0.0013,  0.0132])

In [27]:
prompts[1,:5,0]

tensor([-0.0017, -0.0397,  0.0341,  0.0196, -0.0152])

In [36]:
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer

In [42]:
hmdb51=[i.strip().split(',')[1] for i in open('../lists/hmdb51_labels.txt').readlines()]

In [45]:
_tokenizer=_Tokenizer()

In [46]:
_tokenizer.encode('brush hair')

[8594, 2225]

In [43]:
hmdb51

['brush hair',
 'cartwheel',
 'catch',
 'chew',
 'clap',
 'climb',
 'climb stairs',
 'dive',
 'draw sword',
 'dribble',
 'drink',
 'eat',
 'fall floor',
 'fencing',
 'flic flac',
 'golf',
 'handstand',
 'hit',
 'hug',
 'jump',
 'kick',
 'kick ball',
 'kiss',
 'laugh',
 'pick',
 'pour',
 'pullup',
 'punch',
 'push',
 'pushup',
 'ride bike',
 'ride horse',
 'run',
 'shake hands',
 'shoot ball',
 'shoot bow',
 'shoot gun',
 'sit',
 'situp',
 'smile',
 'smoke',
 'somersault',
 'stand',
 'swing baseball',
 'sword',
 'sword exercise',
 'talk',
 'throw',
 'turn',
 'walk',
 'wave']

In [7]:
s='abca'
for i in s:
    index=ord(i)-ord('a')
    n=chr(index)+'a'
    print(index,n)

0  a
1 a
2 a
0  a


In [8]:
ord('a')

97